# Implemenation: Grid Fitting using Bayesian Statistics w/ PICASO

If this is your first crack at fitting parameters with PICASO, we strong encourage you to look at the first retrieval tutorial. Here we will expand on that tutorial and you will learn: 

**What you will learn:** 

1. How to load a grid and fit parameters using pre-computed spectra
2. How to analyze grid fitting results

**What you should know:**

1. `picaso`'s retrieval class structure (model_set, prior_set, param_set)
2. `picaso.retrieval.setup_retrieval` function to generate template scripts (covered in Retrieval Tutorial 1)

**What you will need:** 

We will build on the `line` fitting tutorial 1, using Grant et al. 2024 data. 

1. Download [WASP-17b grids here](www.doi.org/10.5281/zenodo.14681144) (and unpacked it is 436 Mb)



In [ ]:
import numpy as np
import os 
import picaso.justdoit as jdi
import picaso.analyze as lyz
import xarray as xr
import matplotlib.pyplot as plt

## Step 1) Develop function to get data

(same as tutorial 1) 

Let's create a function to pull our data where all we need to do is declare who the data is from and it pulls it for us automatically.

Note: this format is only a recommendation and you can change any part of this to fit your needs 

In [ ]:
def get_data(): 
    """
    Create a function to process your data in any way you see fit.
    Here we are using the ExoTiC-MIRI data 
    https://zenodo.org/records/8360121/files/ExoTiC-MIRI.zip?download=1
    But no need to download it.

    Checklist
    ---------
    - your function returns a spectrum that will be in the same units as your picaso model (e.g. rp/rs^2, erg/s/cm/cm or other) 
    - your function retuns a spectrum that is in ascending order of wavenumber 
    - your function returns a dictionary where the key specifies the instrument name (in the event there are multiple)

    Returns
    -------
    dict: 
        dictionary key: wavenumber (ascneding), flux or transit depth, and error.
        e.g. {'MIRI LRS':[wavenumber, transit depth, transit depth error], 'NIRSpec G395H':[wavenumber, transit depth, transit depth error]}
    """
    dat = xr.load_dataset(jdi.w17_data())
    #build nice dataframe so we can easily 
    final = jdi.pd.DataFrame(dict(wlgrid_center=dat.coords['central_wavelength'].values,
                transit_depth=dat.data_vars['transit_depth'].values,
                transit_depth_error=dat.data_vars['transit_depth_error'].values))

    #create a wavenumber grid 
    final['wavenumber'] = 1e4/final['wlgrid_center']

    #always ensure we are ordered correctly
    final = final.sort_values(by='wavenumber').reset_index(drop=True)

    #return a nice dictionary with the info we need 
    returns = {'MIRI_LRS': [final['wavenumber'].values, 
             final['transit_depth'].values  ,final['transit_depth_error'].values]   }
    return returns

## Step 2) Load Grid 

Let's point towards the grid locations, create a grid fitter object for them, and prep them. 

If you are not familiar with `lyz.GridFitter` we encourate you to first become familiar with non-Bayesian grid fitting based on purely maximum chi-sq values. You can play around with this [Grid Search tutorial here](https://natashabatalha.github.io/picaso/notebooks/fitdata/GridSearch.html). 

The basic premise of `prep_gridtrieval`:
- vets and transforms the grid to ensure it's square and interprelate-able
- checks that there is a common pressure grid for the temperature

In [ ]:
grid_location = '/data2/models/WASP-17b/spec/zenodo/v1'# should ultimately point to location of all .nc files
grid_name = 'cldfree' #for your own book-keeping
fitter = lyz.GridFitter(grid_name,grid_location, verbose=True, to_fit='transit_depth', save_chem=True)
fitter.prep_gridtrieval(grid_name)

## Step 3) Param Set

In [ ]:
#we can get the grid parameters directly from the module load, this will help with setting up our free parameters
grid_parameters_unique = fitter.interp_params[grid_name]['grid_parameters_unique']

class param_set:
    """
    This is much easier now since it is the grid parameters plus an offset to account for unknown reference pressure
    """
    grid = list(grid_parameters_unique.keys())+['offset']

## Step 4) Guesses Set

In testing, it is very useful to check that it is grabbing the right parameters before doing a full analysis. This is available  for a sanity check if desired

In [ ]:
class guesses_set: 
    """
    For our guesses, we can verify things are working by just taking the first instance of each grid point
    """
    #completely random guesses just to make sure it runs
    grid=[grid_parameters_unique[i][0]
             for i in grid_parameters_unique.keys()] + [0]

## Step 5) Model Set

Let's set up our grid interpolator using picaso's custom interp 

In [ ]:
class model_set:
    """
    There are several different ways to interpolate onto a grid. Here, we use picaso's fast custom interpolator that 
    uses a flex linear interpolator and can be used on any type of matrix grid. 
    """     
    def grid(cube): 
        """Simple grid interpolation + offset for spectra
        """
        final_goal = cube[0:len(grid_parameters_unique.keys())]
        spectra_interp = lyz.custom_interp(final_goal, fitter, grid_name)
        micron = fitter.wavelength[grid_name]
        wno = 1e4/fitter.wavelength[grid_name] 
        offset={'MIRI_LRS':cube[-1]} #remember before it helps always to have the same model returns
        error_inf={} # let's not add error inf 
        return wno, spectra_interp,offset,error_inf

## Step 6) Prior Set

Finally, we are storing all the priors for Ultranest to use.

In [ ]:
class prior_set:
    """
    Store all your priors. You should have the same exact function names in here as
    you do in model_set and param_set

    Make sure the order of the unpacked cube follows the unpacking in your model 
    set and in your parameter set. 
    #pymultinest: http://johannesbuchner.github.io/pymultinest-tutorial/example1.html
    """   
    def grid(cube):
        params = cube.copy()
        for i,key in enumerate(grid_parameters_unique): 
            minn = np.min(grid_parameters_unique[key]) 
            maxx = np.max(grid_parameters_unique[key]) 
            params[i] = minn + (maxx-minn)*params[i]
        #the offset parameter here let's allow these spectra to move up and down by ~1000pm. This is a guess so let's check firts! 
        minn=.008;maxx=.015;
        i+=1;params[i] = minn + (maxx-minn)*params[i]
        return params

## Step 7) Loglikelihood

No changes from our simple line example. 

In [ ]:
def loglikelihood(cube):
    """
    Log_likelihood function that ultimately is given to the sampler
    Note if you keep to our same formats you will not have to change this code move 

    Tips
    ----
    - Remember how we put our data dict, error inflation, and offsets all in dictionary format? Now we can utilize that 
    functionality if we properly named them all with the right keys! 

    Checklist
    --------- 
    - ensure that error inflation and offsets are incorporated in the way that suits your problem 
    - note there are many different ways to incorporate error inflation! this is just one example 
    """
    #compute model spectra
    resultx,resulty,offset_all,err_inf_all = MODEL(cube) # we will define MODEL below 

    #initiate the four terms we willn eed for the likelihood
    ydat_all=[];ymod_all=[];sigma_all=[];extra_term_all=[];

    #loop through data (if multiple instruments, add offsets if present, add err inflation if present)
    for ikey in DATA_DICT.keys(): #we will also define DATA_DICT below
        xdata,ydata,edata = DATA_DICT[ikey]
        xbin_model , y_model = jdi.mean_regrid(resultx, resulty, newx=xdata)#remember we put everything already sorted on wavenumber

        #add offsets if they exist to the data
        offset = offset_all.get(ikey,0) #if offset for that instrument doesnt exist, return 0
        ydata = ydata+offset

        #add error inflation if they exist
        err_inf = err_inf_all.get(ikey,0) #if err inf term for that instrument doesnt exist, return 0
        sigma = edata**2 + (err_inf)**2 #there are multiple ways to do this, here just adding in an extra noise term
        if err_inf !=0: 
            #see formalism here for example https://emcee.readthedocs.io/en/stable/tutorials/line/#maximum-likelihood-estimation
            extra_term = np.log(2*np.pi*sigma)
        else: 
            extra_term=sigma*0

        ydat_all.append(ydata);ymod_all.append(y_model);sigma_all.append(sigma);extra_term_all.append(extra_term); 

    ymod_all = np.concatenate(ymod_all)    
    ydat_all = np.concatenate(ydat_all)    
    sigma_all = np.concatenate(sigma_all)  
    extra_term_all = np.concatenate(extra_term_all)

    #compute likelihood
    loglike = -0.5*np.sum((ydat_all-ymod_all)**2/sigma_all + extra_term_all)
    return loglike

## Step 7) Check models, likelihoods, priors! 

It looks like our priors are giving us an offset that is evenly distributed about the data. Looks good! 

In [ ]:
#we can easity grab all the important pieces now that they are neatly stored in a class structure 
DATA_DICT = get_data()
PARAMS = getattr(param_set,'grid')
MODEL = getattr(model_set,'grid')
PRIOR = getattr(prior_set,'grid')
GUESS = getattr(guesses_set,'grid')

In [ ]:

plt.figure()
#lets plot the data 
for ikey in DATA_DICT.keys(): 
    plt.errorbar(x=DATA_DICT[ikey][0], y=DATA_DICT[ikey][1], yerr=DATA_DICT[ikey][2],color='black',
                 marker='o', ls=' ',label='Grant data')

ntests = 10 #lets do 10 random tests 
for i in range(ntests): 
    cube = np.random.uniform(size=len(PARAMS))
    params_evaluations = PRIOR(cube)
    x,y,off,err = MODEL(params_evaluations)
    loglike = loglikelihood(params_evaluations)
    #REMEMBER: we are adding an offset parameter in the LIKELIHOOD 
    #so when you check you have to make sure that your 
    #offset is included below 
    plt.plot(x,y-params_evaluations[-1],label=str(i)+str(int(loglike)))

guessx,guessy,off,err = MODEL(GUESS)
guess_log = loglikelihood(GUESS)
plt.plot(guessx,guessy-GUESS[-1],color='black',label='guess '+ str(int(guess_log)))
plt.xlim([1e4/14,1e4/5])
plt.legend()

The models that are chosen look randomly distributed around the data. Looks like there are no issues with our sampler. Let's go ahead and run ultranest. 

## Step 8) Run the statistical sampler!! 

This will take a little longer than the simple line since we have more parameters now. But the basics are still identical. 

Some things to notice while the sampler is running: 

1. The ultranest visualization (purple) will show you where the solution is headed towards. It looks like (for example) M/H is immediately drawn toward higher values.
2. For parameters like the offset, where we have manually set the prior, make sure that you are seeing a solution which is not going off the bounds of the prior. In this case, it looks like it is converging right at the median of those two values.

In [ ]:
import ultranest

In [ ]:
sampler = ultranest.ReactiveNestedSampler(PARAMS, loglikelihood, PRIOR,resume=True,log_dir='/data/test/ultranest/grid')
#note if you wanted to turn thsi in the notebook and save the output you would add resume and log_dir above to save
result = sampler.run()

## Analyze Results

In [ ]:
from ultranest.plot import PredictionBand

plt.figure()
first =True
for params in result['samples']:
    x,y,off,err = MODEL(params)
    if first:
        band = PredictionBand(1e4/x);first=False   #transforming xaxis to microns 
    
    #this is a little tricky as we have added an offset to the data 
    #so what ill do in this case is add the inverse to the model so we can visualise it with the data in the banded solution
    band.add(y-params[-1])

band.line(color='black')#median model

#lets plot the 1, 2, and 3 sigma confidence interval from these samples 
for q ,key in zip([k/100/2 for k in [68.27, 95.45, 99.73]], ['1sig','2sig','3sig']): 
        band.shade(q=q,color='grey', alpha=0.2)

for ikey in DATA_DICT.keys(): 
    plt.errorbar(x=1e4/DATA_DICT[ikey][0], y=DATA_DICT[ikey][1], yerr=DATA_DICT[ikey][2], color='red',marker='o', ls=' ',label='Grant data')
    
plt.xlabel('micron')
plt.ylabel('transit depth');

# Short cut to get grid fitting retrieval template in script form

Same as before but now we will add `grid_kwargs` to set some additional features in the `GridFitter`. Note this is only optional and you are always free to change the template manually. 

In [ ]:
import picaso.retrieval as pr

In [ ]:
rtype='grid' #first lets specify the retrieval type 'grid' 
sript_name='run_test.py' #speciy a script name 
sampler_output='/data/test/ultranest/grid'

grid_location = '/data2/models/WASP-17b/spec/zenodo/v1'# should ultimately point to location of all .nc files
grid_name = 'cldfree' #for your own book-keeping
to_fit = 'transit_depth' #this is based on what you want to fit in the xarray files s

#new!
grid_kwargs={'grid_location':grid_location,'grid_name':grid_name,'to_fit':to_fit}

pr.create_template(rtype,sript_name,sampler_output,grid_kwargs=grid_kwargs)

Open up `run_test.py` and modify what you need. We have marked key areas you might want to modify with "CHANGEME"

Running with mpiexec with 5 cpu: 

    >> mpiexec -n 5 python -m mpi4py run_test.py

## Some tricks and tips for once you have a built a script

Now that you have a built script we can input it to grab our built models, priors, parameters set, etc. 

In [ ]:
import run_test as rt #YOUR OWN unique script 

Now you can easily grab what you need directly and no need to have this in a notebook. 

In [ ]:
DATA_DICT = rt.get_data()
PARAMS = getattr(rt.param_set,'grid')
MODEL = getattr(rt.model_set,'grid')
PRIOR = getattr(rt.prior_set,'grid')
GUESS = getattr(rt.guesses_set,'grid')

We can also see use the fitter class to continue our analysis (which we will do in the following section)

# PICASO Analysis Tools from Saved Samples 

This part relies on having ran the above script for the run_test.py and having output in a directory defined by `sampler_output`
## Auto read ultranest results

This function gives you back some of the most highly used output products: 

- `samples_equal`: equally weighted samples from the posterior
- `max_logl` : the maximum loglikelihood (otherwise known as the Bayesian evidence)
- `max_logl_point` : the set of parameters that is associated with the maximum loglikelihood
- `med_intervals` : the 1 sigma median constraint intervals for each of the parameters of interest
- `ultranest_out` : the raw ultranest output 

In [ ]:
results = pr.get_info(sampler_output, PARAMS)

In [ ]:
results.keys()

## Get spectra interval bands and evaluate max loglikelihood spectra  

- `bands_spectra`: 1,2,3 sigma bands for spectra (contains keys such as `1sig_lo` and `1sig_hi` for the spectra)
- `max_logl_spectra`: the spectra evaluated at the max logl point
- `max_logl_error_inflation` : if exists, the error inflation associated with the max logl point
- `max_logl_offsets` : If exists, the offsets associated with the max logl point
- `wavelength`: the wavelength grid in um 

In [ ]:
n_draws=200 #number of evaluations for which to 
evaluations = pr.get_evaluations(results['samples_equal'], results['max_logl_point'], MODEL, n_draws,
                                 regrid=100.0,#spectral resolution to regrid to
                                 pressure_bands=[])#'temperature','H2O','CO2','CO','CH4']) 

In [ ]:
evaluations.keys()

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(7,5))
color_scale =  pr.pals.Blues[3]
resolution=100
f=pr.plot_spectra_bands(evaluations,color_scale, ax=axs,R=resolution)
axs.set_title('Cld Free Model')#add other styles here 
axs.set_xlim([5,14])

## Get Reduced Chi Squared Statistic of Max Logl spectrum Incl Offsets

- `wavenumber`: new regridded wavenumber grid on data axis
- `model` : model regridded on data axis
- `datay` : data with offset included
- `datae` : data error (no error inflation included )
- `chisq_per_datapt` : chi squared per data point (DOF=len of data array)

In [ ]:
chi_info = pr.get_chisq_max(evaluations, DATA_DICT)

In [ ]:
chi_info.keys()

In [ ]:
plt.plot(1e4/chi_info['wavenumber'], chi_info['model'],color='red',label='max logl model')
plt.errorbar(x=1e4/chi_info['wavenumber'], y=chi_info['datay'], yerr=chi_info['datae'], 
             color='black',marker='o', ls=' ')
    

## Get all bundled results in xarray format

What does `data_output` do? 

1. bundles all the median and sigma banded output data into an xarray
2. adds all the constraint intervals to your xarray in latex format
3. creates some default plots of banded spectra (banded chem if you have created it) and corner plots
4. creates a pickle of your equally weighted samples if you have requested it

In [ ]:
filename = '/data/test/output/tagname'#name of file (without extension!!) where you want the output returned 
return_samples=True # do you want a pickle file created of all the equally weighted samples? 
spectrum_tag = 'transit_depth'#in the xarray what do you want the spectrum called? 
spectrum_unit= 'cm**2/cm**2'#what are the units of your spectrum? 
author = "NE Batalha"#who did the analysis? this is for the xarray
contact="natasha.e.batalha@nasa.gov"#how can you be contacted? this is for the xarray
model_description="cloud free grid fit"#describe your model so people will know what it is 
code = "PICASO,Ultranest" # what codes did you use for this analysis? 

bxr=pr.data_output(evaluations, results, chi_info, filename,return_samples=True,
                     spectrum_tag=spectrum_tag,spectrum_unit=spectrum_unit,
                    author=author,contact=contact,
                    model_description=model_description,
                    code=code,)#,
                    #round=[2, 1,1,5,3,2,2,1,2,2])

In [ ]:
bxr

In [ ]:
bxr.attrs['intervals_params']
#note if you do not like the round errors here you can adjust what these numbers are rounded to with 
#round kwarg to data_output

## Stylize Plots

Our default plots were not great, let's beautify them a little 

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(7,5))
color_scale =  pr.pals.Blues[3]
resolution=100
f=pr.plot_spectra_bands(evaluations,color_scale, ax=axs,R=resolution)
axs.set_title('Cld Free Model')#add other styles here 
axs.set_xlim([5,14])

In [ ]:
samples = results['samples_equal']

params =  results['param_names']

#create mapper for labels 
pretty_labels={'cto':'C/O',
               'mh':'log M/H [xSolar]',
               'heat_redis':'heat redis.',
               'offset':'O$_\mathrm{LRS}$',
               'tint':r'T$_\mathrm{int}$'}

#create mapper for ranges ?
ranges={'cto':[0.25,1],
        'tint':[200,300],
        'offset':[0.0085,0.0095],
        'heat_redis':[0.5,0.9],
        'mh':[1.3,1.7]}

ints = eval(bxr.attrs['intervals_params'])#get pretty titles 
intervals={i:ints[i] for i in results['param_names']}

f,a=pr.plot_pair(samples,params,pretty_labels=pretty_labels, ranges=ranges,figsize=(15,15),
              intervals=intervals)
#could make additional style change using a here 
f.savefig('/data/test/output/plot_pair.png')